In [25]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV,train_test_split



In [12]:
heart=pd.read_csv("heart.csv")
print(heart.head())
print(heart.shape)

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  
(303, 14)


In [8]:
'''age : 나이
sex : 성별(0 : 여성; 1 : 남성)
cp : 가슴통증 유형(0 = 증상 없음 ; 1 = 비전형적 협심증; 2 = 비협심증 통증; 3 = 전형적인 협심증)
trestbps : 평시 혈압
chol : 혈중 콜레스테롤 농도
fbs : 공복시 혈당 > 120mg/dl(0 = False; 1 = True)
restecg : 안정 심전도 결과(0 = 좌심실 비대; 1 = 정상; 2 = ST-T파 비정상)
thalach : 최대 심박수
exang : 협심증 유발 운동(0 = no; 1 = yes)
old peak : 비교적 안정되기까지 운동으로 유발되는 ST
slope : 최대 운동 ST segment의 기울기(0 = 하강; 1 = 평면; 2 = 상승)
ca : 형광 투시된 주요 혈관의 수(0~3; 4 = NA)
thal : 0 = NA; 1 = 해결된 결함; 2 = 정상; 3 = 해결가능한 결함
target : 심장병 진단(0 = False; 1 = True)'''

'age : 나이\nsex : 성별(0 : 여성; 1 : 남성)\ncp : 가슴통증 유형(0 = 증상 없음 ; 1 = 비전형적 협심증; 2 = 비협심증 통증; 3 = 전형적인 협심증)\ntrestbps : 평시 혈압\nchol : 혈중 콜레스테롤 농도\nfbs : 공복시 혈당 > 120mg/dl(0 = False; 1 = True)\nrestecg : 안정 심전도 결과(0 = 좌심실 비대; 1 = 정상; 2 = ST-T파 비정상)\nthalach : 최대 심박수\nexang : 협심증 유발 운동(0 = no; 1 = yes)\nold peak : 비교적 안정되기까지 운동으로 유발되는 ST\nslope : 최대 운동 ST segment의 기울기(0 = 하강; 1 = 평면; 2 = 상승)\nca : 형광 투시된 주요 혈관의 수(0~3; 4 = NA)\nthal : 0 = NA; 1 = 해결된 결함; 2 = 정상; 3 = 해결가능한 결함\ntarget : 심장병 진단(0 = False; 1 = True)'

In [17]:
heart.isnull().sum()# null값 없음

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [22]:
heart_data=heart.drop("target",axis=1)# target값만 지정해줌
heart_target=heart["target"]# feature값만 지정해줌

In [26]:
#train_test_split으로 나눠서 예측해본다
dt_clf=DecisionTreeClassifier()
x_train,x_test,y_train,y_test=train_test_split(heart_data,heart_target,
                                               test_size=0.3,random_state=121)
dt_clf.fit(x_train,y_train)
pred=dt_clf.predict(x_test)
print("예측 정확도:{0:.4f}".format(accuracy_score(y_test,pred)))

예측 정확도:0.7143


In [27]:
#예측 정확도가 낮아서 test_size를 조정해서 여러번 실행해봄
size=[0.2,0.3,0.4]
case=0
for i in size:
    case+=1
    x_train,x_test,y_train,y_test=train_test_split(heart_data,heart_target,
                                               test_size=i,random_state=121)
    dt_clf.fit(x_train,y_train)
    pred=dt_clf.predict(x_test)
    print("{0}의 예측 정확도:{1:.4f}".format(case,accuracy_score(y_test,pred)))
    
#0.2일 경우가 가장 정확도가 높았지만 대체로 낮은 예측률을 보여줌

1의 예측 정확도:0.7869
2의 예측 정확도:0.7143
3의 예측 정확도:0.7131


교차검증

kfold

In [47]:
heart_data=heart.drop("target",axis=1)# target값만 지정해줌
heart_target=heart["target"]# feature값만 지정해줌
heart_data_array=heart_data.values#array로 바꿔줘야 index로 kfold가능
heart_target_array=heart_target.values
dt_clf=DecisionTreeClassifier(random_state=156)
kfold=KFold(n_splits=3)#303개의 행이여서 3으로 잡음
cv_accuracy=[]
case=0

for train_index,test_index in kfold.split(heart_data_array):
    x_train,x_test=heart_data_array[train_index],heart_data_array[test_index]
    y_train,y_test=heart_target_array[train_index],heart_target_array[test_index]
    
    #학습,예측
    dt_clf.fit(x_train,y_train)
    pred=dt_clf.predict(x_test)
    case+=1
    
    #정확도 출력
    accuracy=np.round(accuracy_score(y_test,pred),4)
    train_size=x_train.shape[0]
    test_size=x_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(case, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(case,test_index))
    cv_accuracy.append(accuracy)
#kfold 평균 정확도 계산
print("\n## 평균 정확도:",np.mean(cv_accuracy))


#1 교차 검증 정확도 :0.6436, 학습 데이터 크기: 202, 검증 데이터 크기: 101
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]

#2 교차 검증 정확도 :0.7723, 학습 데이터 크기: 202, 검증 데이터 크기: 101
#2 검증 세트 인덱스:[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201]

#3 교차 검증 정확도 :0.4653, 학습 데이터 크기: 202, 검증 데이